In [ ]:
import pandas as pd
from google.colab import files

def load_csv_file():
    print("Please upload your CSV file.")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded.")
        return None

    filename = next(iter(uploaded))
    print(f"Loading '{filename}' into DataFrame.")
    df = pd.read_csv(filename)

    return df

# Load the uploaded CSV file into a DataFrame
data = load_csv_file()

# Check if the data is loaded successfully
if data is not None:
    print("Data loaded successfully.")
    print(data.head())  # Show the first few rows of the DataFrame
else:
    print("Data loading failed.")

Please upload your CSV file.


Saving wiki_movie_plots_deduped.csv to wiki_movie_plots_deduped.csv
Loading 'wiki_movie_plots_deduped.csv' into DataFrame.
Data loaded successfully.
   Release Year                             Title Origin/Ethnicity  \
0          1901            Kansas Saloon Smashers         American   
1          1901     Love by the Light of the Moon         American   
2          1901           The Martyred Presidents         American   
3          1901  Terrible Teddy, the Grizzly King         American   
4          1902            Jack and the Beanstalk         American   

                             Director Cast    Genre  \
0                             Unknown  NaN  unknown   
1                             Unknown  NaN  unknown   
2                             Unknown  NaN  unknown   
3                             Unknown  NaN  unknown   
4  George S. Fleming, Edwin S. Porter  NaN  unknown   

                                           Wiki Page  \
0  https://en.wikipedia.org/wiki/Kansas_Sal

Explantion we loaded the csv file using the upload button

In [ ]:
import pandas as pd

def process_data(filename, output_filename):
    # Load only the 'Title' and 'Plot' columns from the CSV file
    df = pd.read_csv(filename, usecols=['Title', 'Plot'])

    # Here you can apply any function to the dataframe if needed
    # For example, you could do some data cleaning or processing
    # processed_df = your_function(df)

    # For this example, we'll just use the original dataframe
    processed_df = df

    # Save the processed data to a new CSV file
    processed_df.to_csv(output_filename, index=False)
    print(f"Data processed and saved to '{output_filename}'.")

# Specify the name of the already uploaded file and the output file
input_filename = '/content/wiki_movie_plots_deduped.csv'
output_filename = '/content/processed_movies.csv'

# Process the data and save it to another file
process_data(input_filename, output_filename)

Data processed and saved to '/content/processed_movies.csv'.


Explanation: we have removed the extra columns from the dataset and kept "title" and "plot" columns

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

we have installed the sentence transformer libraries

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from rank_bm25 import BM25Okapi

# Load your dataset
dataset = pd.read_csv('/content/processed_movies.csv')

# Combine 'Title' and 'Plot' into a single text passage
dataset['text'] = dataset['Title'] + ": " + dataset['Plot']

# Convert the dataset to a list of texts for processing
corpus = dataset['text'].tolist()

# Tokenize the corpus for BM25
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Initialize the bi-encoder model and pre-compute corpus embeddings
bi_encoder = SentenceTransformer('nq-distilbert-base-v1')
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True)

# Define the search function
def search(query):
    # Perform BM25 search and get top 100 results as an example
    bm25_scores = bm25.get_scores(query.split(" "))
    bm25_ranking = np.argsort(bm25_scores)[::-1][:100]

    # Semantic search with pre-computed embeddings
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    semantic_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    semantic_ranking = torch.argsort(semantic_scores, descending=True).cpu().numpy()[:100]

    # Combine BM25 and semantic rankings using a reranker (CrossEncoder)
    cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    combined_idxs = np.unique(np.concatenate([bm25_ranking, semantic_ranking]))
    cross_inp = [[query, corpus[idx]] for idx in combined_idxs[:50]]  # Top 50 for reranking to reduce computation
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort and select the top 5 results
    top_results = sorted(zip(combined_idxs, cross_scores), key=lambda x: x[1], reverse=True)[:5]
    return [(corpus[idx], score) for idx, score in top_results]

# Example search query
query = "Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions"
results = search(query)

# Display the top 5 results
for text, score in results:
    print(f"Text: {text}\nScore: {score}\n")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Text: The Savage Innocents: Inuk, an Inuk, kills a priest who rejects his traditional offer of food and his wife's company. Pursued by white policemen, Inuk saves the life of one of them, resulting in a final confrontation in which the surviving cop must decide between his commitment to law enforcement and his gratitude to Inuk.
The film's themes include Inuit survival in the extreme arctic wilderness, as well as their raw existence and struggle to maintain their lifestyle against encroaching civilization.
Score: 0.04700668156147003

Text: Nanook of the North: The documentary follows the lives of an Inuk, Nanook, and his family as they travel, search for food, and trade in the Ungava Peninsula of northern Quebec, Canada. Nanook; his wife, Nyla; and their family are introduced as fearless heroes who endure rigors no other race could survive. The audience sees Nanook, often with his family, hunt a walrus, build an igloo, go about his day, and perform other tasks.
Score: -4.55429458618164

We constructed a model using semantic search to get five relevant movie title and plots based on plot passed as a input query

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
import numpy as np

# Load your data
data = pd.read_csv('/content/processed_movies.csv')

# Combine 'Title' and 'Plot' for indexing
corpus = [f"{row['Title']}: {row['Plot']}" for _, row in data.iterrows()]

# Tokenize the corpus for BM25
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Load the sentence transformer model
model = SentenceTransformer('nq-distilbert-base-v1')

# Encode the corpus using the model
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Function to search the corpus
def search(query: str, top_k: int = 5):
    # BM25 search
    bm25_scores = bm25.get_scores(query.split(" "))
    bm25_ranking = np.argsort(bm25_scores)[::-1][:top_k]

    # Semantic search
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    semantic_ranking = torch.argsort(similarities, descending=True).cpu().numpy()[:top_k]

    # Combine results from BM25 and semantic search
    combined_indices = np.unique(np.concatenate([bm25_ranking, semantic_ranking]))

    # Sort combined indices by BM25 scores to get top results
    combined_scores = bm25_scores[combined_indices]
    top_indices = np.argsort(combined_scores)[::-1][:top_k]

    print("Query:", query)
    print("\nTop 5 most similar movies in the corpus:")
    for idx in top_indices:
        print(data.iloc[combined_indices[idx]]['Title'], "(BM25 Score: {:.4f})".format(bm25_scores[combined_indices[idx]]))

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search(user_query)

Enter your movie plot query: Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions
Query: Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions

Top 5 most similar movies in the corpus:
Papilio Buddha (BM25 Score: 27.8773)
Not with My Wife, You Don't! (BM25 Score: 24.9000)
Uriyadi (BM25 Score: 23.2196)
Ladies Only (BM25 Score: 22.2146)
Being Caribou (BM25 Score: 22.1769)


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : Papilio Buddha,Not with My Wife, You Don't!,Uriyadi ,Ladies Only ,Being Caribou out of these five results, Papilio Buddha- RELEVANT, Not with my wife - IRRELEVANT, The Uriyadi - IRRELEVANT, Being Caribou- Irrelevant, Being Caribou - RELEVANT

Recall@1 = 1/(1+1)= 0.5

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are : Papilio Buddha,Not with My Wife, You Don't!,Uriyadi ,Ladies Only ,Being Caribou For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
import numpy as np

# Load your data
data = pd.read_csv('/content/processed_movies.csv')

# Combine 'Title' and 'Plot' for indexing
corpus = [f"{row['Title']}: {row['Plot']}" for _, row in data.iterrows()]

# Tokenize the corpus for BM25
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Load the sentence transformer model
model = SentenceTransformer('nq-distilbert-base-v1')

# Encode the corpus using the model
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Function to search the corpus
def search(query: str, top_k: int = 5):
    # BM25 search
    bm25_scores = bm25.get_scores(query.split(" "))
    bm25_ranking = np.argsort(bm25_scores)[::-1][:top_k]

    # Semantic search
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    semantic_ranking = torch.argsort(similarities, descending=True).cpu().numpy()[:top_k]

    # Combine results from BM25 and semantic search
    combined_indices = np.unique(np.concatenate([bm25_ranking, semantic_ranking]))

    # Sort combined indices by BM25 scores to get top results
    combined_scores = bm25_scores[combined_indices]
    top_indices = np.argsort(combined_scores)[::-1][:top_k]

    print("Query:", query)
    print("\nTop 5 most similar movies in the corpus:")
    for idx in top_indices:
        print(data.iloc[combined_indices[idx]]['Title'], "(BM25 Score: {:.4f})".format(bm25_scores[combined_indices[idx]]))

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search(user_query)

Enter your movie plot query: Western romance
Query: Western romance

Top 5 most similar movies in the corpus:
Hyderabad Blues (BM25 Score: 13.8661)
Out of This World (BM25 Score: 13.1966)
El Akhar (The Other) (BM25 Score: 12.3279)
Blighty (BM25 Score: 11.7846)
The Big Show (BM25 Score: 10.9925)


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : Hyderabad Blues,out of this world, El akhar,Blightly, The big show out of these five results, Hyderabad Blues - RELEVANT, Out of this world - IRRELEVANT, El akhar - IRRELEVANT, Blightly - Irrelevant, The big show - RELEVANT

 Recall@1 = 1/(1+1)= 0.5

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are : Hyderabad Blues,out of this world, El akhar,Blightly, The big show For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
import numpy as np

# Load your data
data = pd.read_csv('/content/processed_movies.csv')

# Combine 'Title' and 'Plot' for indexing
corpus = [f"{row['Title']}: {row['Plot']}" for _, row in data.iterrows()]

# Tokenize the corpus for BM25
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Load the sentence transformer model
model = SentenceTransformer('nq-distilbert-base-v1')

# Encode the corpus using the model
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Function to search the corpus
def search(query: str, top_k: int = 5):
    # BM25 search
    bm25_scores = bm25.get_scores(query.split(" "))
    bm25_ranking = np.argsort(bm25_scores)[::-1][:top_k]

    # Semantic search
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    semantic_ranking = torch.argsort(similarities, descending=True).cpu().numpy()[:top_k]

    # Combine results from BM25 and semantic search
    combined_indices = np.unique(np.concatenate([bm25_ranking, semantic_ranking]))

    # Sort combined indices by BM25 scores to get top results
    combined_scores = bm25_scores[combined_indices]
    top_indices = np.argsort(combined_scores)[::-1][:top_k]

    print("Query:", query)
    print("\nTop 5 most similar movies in the corpus:")
    for idx in top_indices:
        print(data.iloc[combined_indices[idx]]['Title'], "(BM25 Score: {:.4f})".format(bm25_scores[combined_indices[idx]]))

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search(user_query)

Enter your movie plot query: "Silent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo
Query: "Silent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo

Top 5 most similar movies in the corpus:
Sahara (BM25 Score: 66.1401)
Inspiration (BM25 Score: 63.4763)
Ruby Cairo (BM25 Score: 61.3989)
Medusa: Dare to Be Truthful (BM25 Score: 60.9099)
He Who Gets Slapped (BM25 Score: 60.6370)


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : Sahara,Inspiration,Ruby Cario,Medusa,He who gets slapped out of these five results, Sahara - RELEVANT, Inspiration - IRRELEVANT, Ruby Cario - RELEVANT, Medusa - relevant, He who gets slapped - RELEVANT

 Recall@1 = 1/(1+1)= 0.5

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are :Sahara,Inspiration,Ruby Cario,Medusa,He who gets slapped For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
import numpy as np

# Load your data
data = pd.read_csv('/content/processed_movies.csv')

# Combine 'Title' and 'Plot' for indexing
corpus = [f"{row['Title']}: {row['Plot']}" for _, row in data.iterrows()]

# Tokenize the corpus for BM25
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Load the sentence transformer model
model = SentenceTransformer('nq-distilbert-base-v1')

# Encode the corpus using the model
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Function to search the corpus
def search(query: str, top_k: int = 5):
    # BM25 search
    bm25_scores = bm25.get_scores(query.split(" "))
    bm25_ranking = np.argsort(bm25_scores)[::-1][:top_k]

    # Semantic search
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    semantic_ranking = torch.argsort(similarities, descending=True).cpu().numpy()[:top_k]

    # Combine results from BM25 and semantic search
    combined_indices = np.unique(np.concatenate([bm25_ranking, semantic_ranking]))

    # Sort combined indices by BM25 scores to get top results
    combined_scores = bm25_scores[combined_indices]
    top_indices = np.argsort(combined_scores)[::-1][:top_k]

    print("Query:", query)
    print("\nTop 5 most similar movies in the corpus:")
    for idx in top_indices:
        print(data.iloc[combined_indices[idx]]['Title'], "(BM25 Score: {:.4f})".format(bm25_scores[combined_indices[idx]]))

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search(user_query)

Enter your movie plot query: Comedy film, office disguises, boss's daughter, elopement
Query: Comedy film, office disguises, boss's daughter, elopement

Top 5 most similar movies in the corpus:
Floodtide (BM25 Score: 14.8156)
Class of Nuke 'Em High 3: The Good, the Bad and the Subhumanoid (BM25 Score: 13.7324)
The Milkman (BM25 Score: 13.0447)
Delete My Love (BM25 Score: 12.7846)
The Boy Friend (BM25 Score: 12.6869)


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : Floodtide,class of nuke the milkman,delete my love,The boy friend out of these five results,

Recall@1 = 1/(1+1)= 0.5

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are : Floodtide,class of nuke the milkman,delete my love,The boy friend For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
import numpy as np

# Load your data
data = pd.read_csv('/content/processed_movies.csv')

# Combine 'Title' and 'Plot' for indexing
corpus = [f"{row['Title']}: {row['Plot']}" for _, row in data.iterrows()]

# Tokenize the corpus for BM25
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Load the sentence transformer model
model = SentenceTransformer('nq-distilbert-base-v1')

# Encode the corpus using the model
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Function to search the corpus
def search(query: str, top_k: int = 5):
    # BM25 search
    bm25_scores = bm25.get_scores(query.split(" "))
    bm25_ranking = np.argsort(bm25_scores)[::-1][:top_k]

    # Semantic search
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    semantic_ranking = torch.argsort(similarities, descending=True).cpu().numpy()[:top_k]

    # Combine results from BM25 and semantic search
    combined_indices = np.unique(np.concatenate([bm25_ranking, semantic_ranking]))

    # Sort combined indices by BM25 scores to get top results
    combined_scores = bm25_scores[combined_indices]
    top_indices = np.argsort(combined_scores)[::-1][:top_k]

    print("Query:", query)
    print("\nTop 5 most similar movies in the corpus:")
    for idx in top_indices:
        print(data.iloc[combined_indices[idx]]['Title'], "(BM25 Score: {:.4f})".format(bm25_scores[combined_indices[idx]]))

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search(user_query)

Enter your movie plot query: "Lost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria
Query: "Lost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria

Top 5 most similar movies in the corpus:
Cleopatra (BM25 Score: 48.7561)
Cleopatra (BM25 Score: 48.7561)
Cleopatra (BM25 Score: 46.2703)
Serpent of the Nile (BM25 Score: 41.2549)
Cleopatra (BM25 Score: 38.8077)


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions  out of these five results,  Recall@1 = 1/(1+1)= 0.5
The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are : For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
import numpy as np

# Load your data
data = pd.read_csv('/content/processed_movies.csv')

# Combine 'Title' and 'Plot' for indexing
corpus = [f"{row['Title']}: {row['Plot']}" for _, row in data.iterrows()]

# Tokenize the corpus for BM25
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Load the sentence transformer model
model = SentenceTransformer('nq-distilbert-base-v1')

# Encode the corpus using the model
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Function to search the corpus
def search(query: str, top_k: int = 5):
    # BM25 search
    bm25_scores = bm25.get_scores(query.split(" "))
    bm25_ranking = np.argsort(bm25_scores)[::-1][:top_k]

    # Semantic search
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    semantic_ranking = torch.argsort(similarities, descending=True).cpu().numpy()[:top_k]

    # Combine results from BM25 and semantic search
    combined_indices = np.unique(np.concatenate([bm25_ranking, semantic_ranking]))

    # Sort combined indices by BM25 scores to get top results
    combined_scores = bm25_scores[combined_indices]
    top_indices = np.argsort(combined_scores)[::-1][:top_k]

    print("Query:", query)
    print("\nTop 5 most similar movies in the corpus:")
    for idx in top_indices:
        print(data.iloc[combined_indices[idx]]['Title'], "(BM25 Score: {:.4f})".format(bm25_scores[combined_indices[idx]]))

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search(user_query)

Enter your movie plot query: Denis Gage Deane-Tanner
Query: Denis Gage Deane-Tanner

Top 5 most similar movies in the corpus:
Captain Alvarez (BM25 Score: 25.8007)
Indecent Proposal (BM25 Score: 17.0718)
I Love You, Beth Cooper (BM25 Score: 15.9906)
Yolki 6 (BM25 Score: 14.5183)
The Strange Door (BM25 Score: 14.0004)


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : capatain Alvarez,indecent proposal,ilove you, yolki, The stranger door out of these five results,
 Recall@1 = 1/(1+2)= 0.33
The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are :capatain Alvarez,indecent proposal,ilove you, yolki, The stranger door out of these five results, For this query, the reciporcal rank is 1/3 and MRR=3

Now with Semantic Search and Bm25 Model. We are also implementing Re Rank model to combine the results

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from rank_bm25 import BM25Okapi
import numpy as np
import torch

# Load your data
data = pd.read_csv('/content/processed_movies.csv')

# Combine 'Title' and 'Plot' for indexing
corpus = [f"{row['Title']}: {row['Plot']}" for _, row in data.iterrows()]

# Tokenize the corpus for BM25
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Load the bi-encoder model
bi_encoder = SentenceTransformer('nq-distilbert-base-v1')
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True)

# Load the cross-encoder model for reranking
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Function to search and rerank the corpus
def search_and_rerank(query: str, top_k: int = 5):
    # Perform BM25 search to get a broader set of candidates
    bm25_scores = bm25.get_scores(query.split(" "))
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:100]

    # Semantic search to refine the candidates
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    semantic_top_idx = torch.argsort(similarities, descending=True).cpu().numpy()[:100]

    # Combine results from BM25 and semantic search for reranking
    combined_idx = np.unique(np.concatenate([bm25_top_idx, semantic_top_idx]))

    # Prepare the reranking input and perform cross-encoder reranking
    rerank_corpus = [corpus[idx] for idx in combined_idx]
    rerank_input = [[query, passage] for passage in rerank_corpus]
    rerank_scores = cross_encoder.predict(rerank_input)

    # Select the top results after reranking
    top_indices = np.argsort(rerank_scores)[::-1][:top_k]

    print("Query:", query)
    print("\nTop 5 most relevant movies:")
    for idx in top_indices:
        original_idx = combined_idx[idx]
        print(data.iloc[original_idx]['Title'])

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search_and_rerank(user_query)

Enter your movie plot query: Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions
Query: Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions

Top 5 most relevant movies:
The Savage Innocents
The Last Wave
Nanook of the North
Searchers (Maliglutit)
The White Dawn


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : The savage innocents, the last wave, nanook of the north,searchers,the white dawn out of these five results, The savage innocents- RELEVANT, The last wave - IRRELEVANT, searchers - RELEVANT,Nanook of north- Irrelevant, The white dawn- IRRELEVANT

 Recall@1 = 1/(1+2)= 0.33

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are :The savage innocents, the last wave, nanook of the north,searchers,the white dawn For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from rank_bm25 import BM25Okapi
import numpy as np
import torch

# Load your data
data = pd.read_csv('/content/processed_movies.csv')

# Combine 'Title' and 'Plot' for indexing
corpus = [f"{row['Title']}: {row['Plot']}" for _, row in data.iterrows()]

# Tokenize the corpus for BM25
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Load the bi-encoder model
bi_encoder = SentenceTransformer('nq-distilbert-base-v1')
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True)

# Load the cross-encoder model for reranking
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Function to search and rerank the corpus
def search_and_rerank(query: str, top_k: int = 5):
    # Perform BM25 search to get a broader set of candidates
    bm25_scores = bm25.get_scores(query.split(" "))
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:100]

    # Semantic search to refine the candidates
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    semantic_top_idx = torch.argsort(similarities, descending=True).cpu().numpy()[:100]

    # Combine results from BM25 and semantic search for reranking
    combined_idx = np.unique(np.concatenate([bm25_top_idx, semantic_top_idx]))

    # Prepare the reranking input and perform cross-encoder reranking
    rerank_corpus = [corpus[idx] for idx in combined_idx]
    rerank_input = [[query, passage] for passage in rerank_corpus]
    rerank_scores = cross_encoder.predict(rerank_input)

    # Select the top results after reranking
    top_indices = np.argsort(rerank_scores)[::-1][:top_k]

    print("Query:", query)
    print("\nTop 5 most relevant movies:")
    for idx in top_indices:
        original_idx = combined_idx[idx]
        print(data.iloc[original_idx]['Title'])

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search_and_rerank(user_query)

Enter your movie plot query: Western romance
Query: Western romance

Top 5 most relevant movies:
El Akhar (The Other)
The Big Show
The Magic of Belle Isle
Saturday's Millions
This Is Not What I Expected


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are :El akhar,The big show,The magic of belle isle,saturdays millions, this is not what i expected out of these five results,

 Recall@1 = 1/(1+1)= 0.5

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are :l aEl akhar,The big show,The magic of belle isle,saturdays millions, this is not what i expected   For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from rank_bm25 import BM25Okapi
import numpy as np
import torch

# Load your data
data = pd.read_csv('/content/processed_movies.csv')

# Combine 'Title' and 'Plot' for indexing
corpus = [f"{row['Title']}: {row['Plot']}" for _, row in data.iterrows()]

# Tokenize the corpus for BM25
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Load the bi-encoder model
bi_encoder = SentenceTransformer('nq-distilbert-base-v1')
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True)

# Load the cross-encoder model for reranking
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Function to search and rerank the corpus
def search_and_rerank(query: str, top_k: int = 5):
    # Perform BM25 search to get a broader set of candidates
    bm25_scores = bm25.get_scores(query.split(" "))
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:100]

    # Semantic search to refine the candidates
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    semantic_top_idx = torch.argsort(similarities, descending=True).cpu().numpy()[:100]

    # Combine results from BM25 and semantic search for reranking
    combined_idx = np.unique(np.concatenate([bm25_top_idx, semantic_top_idx]))

    # Prepare the reranking input and perform cross-encoder reranking
    rerank_corpus = [corpus[idx] for idx in combined_idx]
    rerank_input = [[query, passage] for passage in rerank_corpus]
    rerank_scores = cross_encoder.predict(rerank_input)

    # Select the top results after reranking
    top_indices = np.argsort(rerank_scores)[::-1][:top_k]

    print("Query:", query)
    print("\nTop 5 most relevant movies:")
    for idx in top_indices:
        original_idx = combined_idx[idx]
        print(data.iloc[original_idx]['Title'])

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search_and_rerank(user_query)

Enter your movie plot query: "Silent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo
Query: "Silent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo

Top 5 most relevant movies:
Sahara
He Who Gets Slapped
Death on the Nile
Cairo Time
The Suburbanite


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : Sahara, he who gets slapped, death on the nile, cairo time, The suburbanite out of these five results,

 Recall@1 = 1/(1+1)= 0.5

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are : Sahara, he who gets slapped, death on the nile, cairo time, The suburbanite For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from rank_bm25 import BM25Okapi
import numpy as np
import torch

# Load your data
data = pd.read_csv('/content/processed_movies.csv')

# Combine 'Title' and 'Plot' for indexing
corpus = [f"{row['Title']}: {row['Plot']}" for _, row in data.iterrows()]

# Tokenize the corpus for BM25
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Load the bi-encoder model
bi_encoder = SentenceTransformer('nq-distilbert-base-v1')
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True)

# Load the cross-encoder model for reranking
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Function to search and rerank the corpus
def search_and_rerank(query: str, top_k: int = 5):
    # Perform BM25 search to get a broader set of candidates
    bm25_scores = bm25.get_scores(query.split(" "))
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:100]

    # Semantic search to refine the candidates
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    semantic_top_idx = torch.argsort(similarities, descending=True).cpu().numpy()[:100]

    # Combine results from BM25 and semantic search for reranking
    combined_idx = np.unique(np.concatenate([bm25_top_idx, semantic_top_idx]))

    # Prepare the reranking input and perform cross-encoder reranking
    rerank_corpus = [corpus[idx] for idx in combined_idx]
    rerank_input = [[query, passage] for passage in rerank_corpus]
    rerank_scores = cross_encoder.predict(rerank_input)

    # Select the top results after reranking
    top_indices = np.argsort(rerank_scores)[::-1][:top_k]

    print("Query:", query)
    print("\nTop 5 most relevant movies:")
    for idx in top_indices:
        original_idx = combined_idx[idx]
        print(data.iloc[original_idx]['Title'])

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search_and_rerank(user_query)

Enter your movie plot query: "Comedy film, office disguises, boss's daughter, elopement.
Query: "Comedy film, office disguises, boss's daughter, elopement.

Top 5 most relevant movies:
Bucking Broadway
Mabel's Blunder
The Double
He Hired the Boss
The Bedroom Window


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : Bucking broadway, mabels blunder, the double, he hired the boss, the bedroom window. out of these five results,

 Recall@1 = 1/(1+1)= 0.5

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are :  Bucking broadway, mabels blunder, the double, he hired the boss, the bedroom window For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from rank_bm25 import BM25Okapi
import numpy as np
import torch

# Load your data
data = pd.read_csv('/content/processed_movies.csv')

# Combine 'Title' and 'Plot' for indexing
corpus = [f"{row['Title']}: {row['Plot']}" for _, row in data.iterrows()]

# Tokenize the corpus for BM25
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Load the bi-encoder model
bi_encoder = SentenceTransformer('nq-distilbert-base-v1')
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True)

# Load the cross-encoder model for reranking
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Function to search and rerank the corpus
def search_and_rerank(query: str, top_k: int = 5):
    # Perform BM25 search to get a broader set of candidates
    bm25_scores = bm25.get_scores(query.split(" "))
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:100]

    # Semantic search to refine the candidates
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    semantic_top_idx = torch.argsort(similarities, descending=True).cpu().numpy()[:100]

    # Combine results from BM25 and semantic search for reranking
    combined_idx = np.unique(np.concatenate([bm25_top_idx, semantic_top_idx]))

    # Prepare the reranking input and perform cross-encoder reranking
    rerank_corpus = [corpus[idx] for idx in combined_idx]
    rerank_input = [[query, passage] for passage in rerank_corpus]
    rerank_scores = cross_encoder.predict(rerank_input)

    # Select the top results after reranking
    top_indices = np.argsort(rerank_scores)[::-1][:top_k]

    print("Query:", query)
    print("\nTop 5 most relevant movies:")
    for idx in top_indices:
        original_idx = combined_idx[idx]
        print(data.iloc[original_idx]['Title'])

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search_and_rerank(user_query)

Enter your movie plot query: "Lost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria
Query: "Lost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria

Top 5 most relevant movies:
Cleopatra
Serpent of the Nile
Carry On Cleo
Caesar and Cleopatra
Cleopatra


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are :Cleopatra, serpent of the nile, carry on cleo, caesar and cleopatra, cleopatra out of these five results,

 Recall@1 = 1/(1+2)= 0.33

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are :Cleopatra, serpent of the nile, carry on cleo, caesar and cleopatra, cleopatra For this query, the reciporcal rank is 1/3 and MRR=3

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from rank_bm25 import BM25Okapi
import numpy as np
import torch

# Load your data
data = pd.read_csv('/content/processed_movies.csv')

# Combine 'Title' and 'Plot' for indexing
corpus = [f"{row['Title']}: {row['Plot']}" for _, row in data.iterrows()]

# Tokenize the corpus for BM25
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Load the bi-encoder model
bi_encoder = SentenceTransformer('nq-distilbert-base-v1')
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True)

# Load the cross-encoder model for reranking
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Function to search and rerank the corpus
def search_and_rerank(query: str, top_k: int = 5):
    # Perform BM25 search to get a broader set of candidates
    bm25_scores = bm25.get_scores(query.split(" "))
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:100]

    # Semantic search to refine the candidates
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    semantic_top_idx = torch.argsort(similarities, descending=True).cpu().numpy()[:100]

    # Combine results from BM25 and semantic search for reranking
    combined_idx = np.unique(np.concatenate([bm25_top_idx, semantic_top_idx]))

    # Prepare the reranking input and perform cross-encoder reranking
    rerank_corpus = [corpus[idx] for idx in combined_idx]
    rerank_input = [[query, passage] for passage in rerank_corpus]
    rerank_scores = cross_encoder.predict(rerank_input)

    # Select the top results after reranking
    top_indices = np.argsort(rerank_scores)[::-1][:top_k]

    print("Query:", query)
    print("\nTop 5 most relevant movies:")
    for idx in top_indices:
        original_idx = combined_idx[idx]
        print(data.iloc[original_idx]['Title'])

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search_and_rerank(user_query)

Enter your movie plot query: Denis Gage Deane-Tanner
Query: Denis Gage Deane-Tanner

Top 5 most relevant movies:
Captain Alvarez
I Love You, Beth Cooper
Skin & Bone
Dean
Indecent Proposal


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are :Captain Alvarez, I Love You,Beth Cooper, Skin & bone,Dean,Indecent Proposal out of these five results,

 Recall@1 = 1/(1+1)= 0.5

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are : Captain Alvarez, I Love You,Beth Cooper, Skin & bone,Dean,Indecent Proposal For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)